Compose function - Curve graph

In [ ]:
import torch
import numpy as np
from torch import nn

In [ ]:
class MultiLayerNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
        nn.Linear(1, 128),
        nn.ReLU(),
        nn.Linear(128, 64),
        nn.ReLU(),
        nn.Linear(64, 32),
        nn.ReLU(),
        nn.Linear(32, 8),
        nn.ReLU(),
        nn.Linear(8, 1),
    )

  def forward(self, x):
    return self.layers(x)

In [ ]:
from torch.utils.data import Dataset, DataLoader
import torch.distributions.uniform as urand

In [ ]:
class AlgebraicDataset(Dataset):
  #pass parameters: self, function, interval, sample
  def __init__(self, f, interval, nsamples):
    X = urand.Uniform(interval[0], interval[1]).sample([nsamples])
    self.data = [(x, f(x)) for x in X]

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    return self.data[idx]

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Run on {device}")
#See its run on GPU device

In [ ]:
multimodel = MultiLayerNetwork().to(device)

In [ ]:
from math import cos

In [ ]:
f = lambda x: cos(x/2)

In [ ]:
interval = (-10, 10)
train_nsamples = 1000
test_nsamples = 100

In [ ]:
train_dataset = AlgebraicDataset(f, interval, train_nsamples)
test_dataset = AlgebraicDataset(f, interval, test_nsamples)

train_dataloader = DataLoader(train_dataset, train_nsamples, shuffle=True)
test_dataloader = DataLoader(test_dataset, test_nsamples, shuffle=True)

In [ ]:
lossfunc = nn.MSELoss()
optimizer = torch.optim.SGD(multimodel.parameters(), lr=1e-3)

In [ ]:
def train(model, dataloader, lossfunc, optimizer):
  model.train()
  cumloss = 0.0
  for X, y in dataloader:
    X = X.unsqueeze(1).float().to(device)
    y = y.unsqueeze(1).float().to(device)

    pred = model(X)
    loss = lossfunc(pred, y)

    # clean cache
    optimizer.zero_grad()
    # compute gradients
    loss.backward()
    # reduces local error
    optimizer.step()

    # loss is a tensor; to get float
    cumloss += loss.item() 
  
  return cumloss / len(dataloader)


def test(model, dataloader, lossfunc):
  model.eval()
  
  cumloss = 0.0
  with torch.no_grad():
    for X, y in dataloader:
      X = X.unsqueeze(1).float().to(device)
      y = y.unsqueeze(1).float().to(device)

      pred = model(X)
      loss = lossfunc(pred, y)
      cumloss += loss.item() 
  
  return cumloss / len(dataloader)

In [ ]:
#define amount and show

epochs = 20001
for t in range(epochs):
	train_loss = train(multimodel, train_dataloader, lossfunc, optimizer)
	if t % 100 == 0:
		print(f"Epoch: {t}; Train Loss: {train_loss}")
		plot_comparinson(f, multimodel, nsamples=40)

test_loss = test(multimodel, test_dataloader, lossfunc)
print(f"Test Loss: {test_loss}")

Expected result: 
First epoch step:

![Untitled](https://prod-files-secure.s3.us-west-2.amazonaws.com/3d7f32a3-eddd-46e3-a6ee-4a4f8ffd2aab/7dcf1ab6-f42d-422d-949f-56d68b7da6c6/Untitled.png)

Last epoch step:
![Untitled](https://prod-files-secure.s3.us-west-2.amazonaws.com/3d7f32a3-eddd-46e3-a6ee-4a4f8ffd2aab/c2da6a68-78f3-4c5a-8376-d4f5be144b9a/Untitled.png)